In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.io import read_image
import os



In [7]:
# Define the neural network
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # Adjust the input size based on your actual input size and batch size
        self.fc1 = nn.Linear(32 * 32 * 32, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)
        x = self.fc1(x)
        return x

In [8]:
# Load the dataset
data_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root='F:/PROJECT/Machine Learning/Capstone/datasets/', transform=data_transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [9]:
# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



In [10]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')



Epoch [1/10], Loss: 0.4879298508167267
Epoch [2/10], Loss: 0.48227882385253906
Epoch [3/10], Loss: 0.2710327208042145
Epoch [4/10], Loss: 0.32336267828941345
Epoch [5/10], Loss: 0.14979061484336853
Epoch [6/10], Loss: 0.05613323301076889
Epoch [7/10], Loss: 0.17163769900798798
Epoch [8/10], Loss: 0.03970998525619507
Epoch [9/10], Loss: 0.08158981055021286
Epoch [10/10], Loss: 0.023977814242243767


In [11]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy}%')


Test Accuracy: 97.22222222222221%


In [13]:
# Save the model
torch.save(model.state_dict(), 'F:/PROJECT/Machine Learning/tensor_vs_pytorch/model.pth')
print('Saved PyTorch Model State to model.pth')


Saved PyTorch Model State to model.pth


# test the model

In [15]:
import torch
from torchvision import transforms
from PIL import Image

# Assuming you've already trained and saved your model
model = SimpleCNN()
model.load_state_dict(torch.load('F:/PROJECT/Machine Learning/tensor_vs_pytorch/model.pth'))
model.eval()


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=32768, out_features=2, bias=True)
)

In [20]:
# Define the transform for the input image
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load and preprocess the local image
image_path = "C:/Users/haica/Downloads/ikan2.png"  # Replace with the path to your local image
image = Image.open(image_path).convert("RGB")
input_image = test_transform(image).unsqueeze(0)  # Add a batch dimension



In [21]:
with torch.no_grad():
    model.eval()
    output = model(input_image)

# Get the predicted class
_, predicted_class = torch.max(output, 1)
print(f'Predicted Class: {predicted_class.item()}')


Predicted Class: 0
